In [2]:
import requests
import json

# Replace these with your Redash instance details
REDASH_HOST = "https://redash.truckitin.ai"
API_KEY = "loMLO5S6tcTusPWt5dcExEA4qMaRRQBbrkbcSuLx"

# URL for the data sources API
url = f"{REDASH_HOST}/api/data_sources"

# Headers with the API key
headers = {"Authorization": f"Key {API_KEY}"}

# Make the request
response = requests.get(url, headers=headers)

# Check for a successful response
if response.status_code == 200:
    # Parse the JSON response
    data_sources = response.json()
    print(json.dumps(data_sources, indent=2))
else:
    print(f"Failed to get data sources: {response.status_code} - {response.text}")

[
  {
    "name": "Amigu-Production",
    "pause_reason": null,
    "syntax": "sql",
    "paused": 0,
    "view_only": false,
    "type": "rds_mysql",
    "id": 18
  },
  {
    "name": "Billing Production",
    "pause_reason": null,
    "syntax": "sql",
    "paused": 0,
    "view_only": false,
    "type": "rds_mysql",
    "id": 8
  },
  {
    "name": "Datawarehouse",
    "pause_reason": null,
    "syntax": "sql",
    "paused": 0,
    "view_only": false,
    "type": "rds_mysql",
    "id": 6
  },
  {
    "name": "Dimagh Production",
    "pause_reason": null,
    "syntax": "sql",
    "paused": 0,
    "view_only": false,
    "type": "rds_mysql",
    "id": 2
  },
  {
    "name": "Ledger Production",
    "pause_reason": null,
    "syntax": "sql",
    "paused": 0,
    "view_only": false,
    "type": "rds_mysql",
    "id": 4
  },
  {
    "name": "rider_db_notification",
    "pause_reason": null,
    "syntax": "sql",
    "paused": 0,
    "view_only": false,
    "type": "mysql",
    "id": 15
  }

In [3]:
import requests
import time
import pandas as pd


def run_query(api_key, query, data_source_id):
    base_url = "https://redash.truckitin.ai/api"
    headers = {"Authorization": f"Key {api_key}"}

    # Create and execute the query
    response = requests.post(
        f"{base_url}/queries",
        headers=headers,
        json={"query": query, "data_source_id": data_source_id},
    )
    query_id = response.json()["id"]

    # Execute the query
    response = requests.post(f"{base_url}/queries/{query_id}/results", headers=headers)
    job = response.json()["job"]

    # Poll for query results
    while job["status"] not in (3, 4):  # 3: completed, 4: failed
        response = requests.get(f'{base_url}/jobs/{job["id"]}', headers=headers)
        job = response.json()["job"]
        time.sleep(1)

    if job["status"] == 3:
        results = requests.get(
            f"{base_url}/queries/{query_id}/results.json", headers=headers
        ).json()["query_result"]["data"]
        return pd.DataFrame(results["rows"])
    else:
        raise Exception("Query execution failed.")


# Usage example
api_key = "loMLO5S6tcTusPWt5dcExEA4qMaRRQBbrkbcSuLx"
data_source_id = 14  # You need to know this beforehand
query = "SELECT * FROM OrderDetails LIMIT 10"

df = run_query(api_key, query, data_source_id)
print(df)

KeyError: 'id'

In [8]:
import requests
import json
import pandas as pd
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

REDASH_HOST = "https://redash.truckitin.ai"
API_KEY = "loMLO5S6tcTusPWt5dcExEA4qMaRRQBbrkbcSuLx"
DATA_SOURCE_ID = "14"

# Ad-hoc SQL query
query = "SELECT * FROM OrderDetails LIMIT 10"

# URL for running the query
url = f"{REDASH_HOST}/api/query_results"

# Headers with the API key
headers = {"Authorization": f"Key {API_KEY}", "Content-Type": "application/json"}

# Payload with the data source ID and query
payload = {"data_source_id": DATA_SOURCE_ID, "query": query}


def run_query(url, headers, payload):
    try:
        # Make the request
        response = requests.post(url, headers=headers, data=json.dumps(payload))

        # Check for a successful response
        response.raise_for_status()

        # Parse the JSON response
        result = response.json()

        # Extract the query results
        query_results = result["query_result"]["data"]["rows"]

        # Convert the query results to a Pandas DataFrame
        df = pd.DataFrame(query_results)

        return df

    except requests.exceptions.RequestException as e:
        logging.error(f"Request failed: {e}")
        return None
    except KeyError as e:
        logging.error(f"Unexpected response format: {e}")
        return None


# Run the query and get the DataFrame
df = run_query(url, headers, payload)

if df is not None:
    logging.info("Query executed successfully.")
    # Proceed with further processing on df
    print(df.head())
else:
    logging.error("Query execution failed.")

2024-08-05 12:42:13,988 - INFO - Query executed successfully.


   transit_received  area_id  city_id  delivery_type_id  shipping_term_id  \
0                 1     6209      772                 2                 1   
1                 0     2012        1                 2                 1   
2                 1     6209      772                 2                 1   
3                 0      364        2                 2                 1   
4                 1      561        3                 2                 1   

   order_status_id_prev sort_addr_id province_name_dest_city  \
0                   8.0         None                  Punjab   
1                   8.0         None                   Sindh   
2                  18.0         None                  Punjab   
3                  18.0         None                  Punjab   
4                  18.0         None                   Sindh   

   payment_mode_rider  call_origin  ...  payment_mode_customer_title  \
0                 2.0          203  ...                         None   
1       